# 1. `Customers_With_More_Than_10_Orders.sql`

## Proposition
This query retrieves customers who have placed more than 10 orders, displaying their `CustomerID`, `CustomerName`, and the total number of orders.

## Tables
- `Sales.Customers`: Contains customer details.
- `Sales.Orders`: Contains information about orders placed by customers.

## Columns
- `c.CustomerID`: Unique identifier for each customer from the `Sales.Customers` table.
- `c.CustomerName`: Name of the customer from the `Sales.Customers` table.
- `CustomerOrderCounts.CustomerID`: The identifier of customers with order counts, retrieved from a subquery that counts the number of orders per customer in the `Sales.Orders` table.
- `CustomerOrderCounts.TotalOrders`: The total number of orders for each customer, filtered to show only those with more than 10 orders.

In [ ]:
USE WideWorldImporters

SELECT c.CustomerID, c.CustomerName, TotalOrders
FROM Sales.Customers c
JOIN (
    SELECT CustomerID, COUNT(OrderID) AS TotalOrders
    FROM Sales.Orders
    GROUP BY CustomerID
) AS CustomerOrderCounts ON c.CustomerID = CustomerOrderCounts.CustomerID
WHERE TotalOrders > 10;

___
___
___

# 2. `Latest_Customer_Orders.sql`

## Proposition
This query retrieves the most recent order for each customer, displaying their `CustomerID`, `CustomerName`, `OrderID`, and `OrderDate`.

## Tables
- `Sales.Customers`: Contains customer details.
- `Sales.Orders`: Contains information about customer orders.

## Columns
- `c.CustomerID`: Unique identifier for each customer from the `Sales.Customers` table.
- `c.CustomerName`: Name of the customer from the `Sales.Customers` table.
- `LastOrders.CustomerID`: The identifier of customers who have made orders, retrieved from a subquery that selects the latest order date for each customer.
- `LastOrders.LastOrderDate`: The most recent order date for each customer, from the subquery.
- `o.OrderID`: The unique identifier of the most recent order from the `Sales.Orders` table.
- `o.OrderDate`: The date of the most recent order from the `Sales.Orders` table.


In [ ]:
USE WideWorldImporters

SELECT c.CustomerID, CustomerName, OrderID, OrderDate
FROM Sales.Customers c
JOIN (
    SELECT CustomerID, MAX(OrderDate) AS LastOrderDate
    FROM Sales.Orders
    GROUP BY CustomerID
) AS LastOrders ON c.CustomerID = LastOrders.CustomerID
JOIN Sales.Orders o ON o.CustomerID = LastOrders.CustomerID AND o.OrderDate = LastOrders.LastOrderDate;


___
___
___

# 3. `StockItems_Not_Ordered.sql`

## Proposition
This query retrieves stock items that have never been ordered, displaying their `StockItemID` and `StockItemName`.

## Tables
- `Warehouse.StockItems`: Contains details about stock items available in the warehouse.
- `Sales.OrderLines`: Contains information about items included in customer orders.

## Columns
- `si.StockItemID`: Unique identifier for each stock item from the `Warehouse.StockItems` table.
- `si.StockItemName`: Name of the stock item from the `Warehouse.StockItems` table.
- `ol.StockItemID`: The identifier of stock items that have been ordered, used in the subquery to filter out stock items that appear in `Sales.OrderLines`.


In [ ]:
USE WideWorldImporters

SELECT StockItemID, StockItemName
FROM Warehouse.StockItems si
WHERE NOT EXISTS (
    SELECT 1
    FROM Sales.OrderLines ol
    WHERE ol.StockItemID = si.StockItemID
);

___
___
___

# 4. `Customers_Total_Order_Above_10000.sql`

## Proposition
This query retrieves customers who have placed orders with a total order value exceeding **$10,000**, displaying their `CustomerID`, `CustomerName`, and `TotalOrderValue`.

## Tables
- `Sales.Customers`: Contains customer details.
- `Sales.Orders`: Contains information about customer orders.
- `Sales.OrderLines`: Contains details about the specific items ordered within each order.

## Columns
- `c.CustomerID`: Unique identifier for each customer from the Sales.Customers table.
- `c.CustomerName`: Name of the customer from the Sales.Customers table.
- `CustomerOrderTotals.CustomerID`: The identifier of customers, used to join with `Sales.Customers` and calculated from the inner subquery.
- `CustomerOrderTotals.TotalOrderValue`: The total value of all orders for each customer, calculated by multiplying the quantity of items ordered by their unit price, and summed across all orders for that customer.
- `ol.Quantity`: The quantity of each item ordered, used in the calculation of the total order value.
- `ol.UnitPrice`: The price of each unit of an item ordered, used in the calculation of the total order value.


In [ ]:
USE WideWorldImporters

SELECT c.CustomerID, CustomerName, TotalOrderValue
FROM Sales.Customers c
JOIN (
    SELECT CustomerID, SUM(Quantity * UnitPrice) AS TotalOrderValue
    FROM Sales.OrderLines ol
    JOIN Sales.Orders o ON ol.OrderID = o.OrderID
    GROUP BY CustomerID
) AS CustomerOrderTotals ON c.CustomerID = CustomerOrderTotals.CustomerID
WHERE TotalOrderValue > 10000;

___
___
___

# 5. `Top_5_Suppliers_By_Products_Supplied.sql`

## Proposition
This query retrieves the top 5 suppliers based on the total number of products they supply, displaying their `SupplierID`, `SupplierName`, and the total number of products supplied.

## Tables
- `Purchasing.Suppliers`: Contains details about suppliers.
- `Warehouse.StockItems`: Contains details about stock items supplied by each supplier.

## Columns
- `s.SupplierID`: Unique identifier for each supplier from the `Purchasing.Suppliers` table.
- `s.SupplierName`: Name of the supplier from the `Purchasing.Suppliers` table.
- `SupplierProductCounts.SupplierID`: The identifier of suppliers, used to join with the `Purchasing.Suppliers` table and retrieved from a subquery that counts the number of stock items supplied by each supplier.
- `SupplierProductCounts.TotalProductsSupplied`: The total number of products supplied by each supplier, calculated by counting the `StockItemID` from the `Warehouse.StockItems` table.
- `StockItemID`: Unique identifier of stock items, used in the subquery to count the total number of products supplied by each supplier.

In [ ]:
USE WideWorldImporters

SELECT s.SupplierID, SupplierName, TotalProductsSupplied
FROM Purchasing.Suppliers s
JOIN (
    SELECT SupplierID, COUNT(StockItemID) AS TotalProductsSupplied
    FROM Warehouse.StockItems
    GROUP BY SupplierID
) AS SupplierProductCounts ON s.SupplierID = SupplierProductCounts.SupplierID
ORDER BY TotalProductsSupplied DESC
OFFSET 0 ROWS FETCH NEXT 5 ROWS ONLY;

___
___
___

# 6. `StockItems_With_Avg_Order_Quantity_Above_50.sql`

## Proposition
This query retrieves stock items with an average order quantity greater than **50**, displaying their `StockItemID`, `StockItemName`, and the average order quantity.

## Tables
- `Warehouse.StockItems`: Contains details about stock items available in the warehouse.
- `Sales.OrderLines`: Contains details about the specific items ordered within each order.

## Columns
- `si.StockItemID`: Unique identifier for each stock item from the `Warehouse.StockItems` table.
- `si.StockItemName`: Name of the stock item from the `Warehouse.StockItems` table.
- `ProductOrderAverages.StockItemID`: The identifier of stock items, used to join with the `Warehouse.StockItems` table and retrieved from a subquery that calculates the average quantity ordered for each stock item.
- `ProductOrderAverages.AvgOrderQuantity`: The average quantity of each item ordered, calculated from the `Sales.OrderLines` table.
- `Quantity`: The number of units ordered for each stock item, used in the subquery to calculate the average order quantity.

In [ ]:
USE WideWorldImporters

SELECT si.StockItemID, StockItemName, AvgOrderQuantity
FROM Warehouse.StockItems si
JOIN (
    SELECT StockItemID, AVG(Quantity) AS AvgOrderQuantity
    FROM Sales.OrderLines
    GROUP BY StockItemID
) AS ProductOrderAverages ON si.StockItemID = ProductOrderAverages.StockItemID
WHERE AvgOrderQuantity > 50;

___
___
___

# 7. `Employees_With_Over_100_Orders.sql`

## Proposition
This query retrieves employees who have processed more than **100** orders, displaying their `EmployeeID`, `FullName`, and the total number of orders they processed.

## Tables
- `Application.People`: Contains personal information about employees.
- `Sales.Orders`: Contains details about customer orders, including the salesperson responsible for each order.

## Columns
- `p.EmployeeID`: Unique identifier for each employee from the `Application.People` table (retrieved as `SalespersonPersonID` in the subquery).
- `p.FullName`: The full name of the employee from the `Application.People` table.
- `EmployeeOrderCounts.EmployeeID`: The identifier of employees who processed orders, retrieved from a subquery that counts the number of orders processed by each salesperson.
- `EmployeeOrderCounts.OrdersProcessed`: The total number of orders processed by each employee, calculated from the `Sales.Orders` table.
- `SalespersonPersonID`: The ID of the employee (salesperson) associated with processing an order, used in the subquery to count the total number of orders for each employee.
- `OrderID`: The unique identifier for each order, used to count the number of orders processed by each employee.

In [ ]:
USE WideWorldImporters

SELECT EmployeeID, FullName, OrdersProcessed
FROM Application.People p
JOIN (
    SELECT SalespersonPersonID AS EmployeeID, COUNT(OrderID) AS OrdersProcessed
    FROM Sales.Orders
    GROUP BY SalespersonPersonID
) AS EmployeeOrderCounts ON p.PersonID = EmployeeOrderCounts.EmployeeID
WHERE OrdersProcessed > 100;

___
___
___

# 8. `Customers_With_No_Transactions.sql`

## Proposition
This query retrieves customers who have never made a transaction, displaying their `CustomerID` and `CustomerName`.

## Tables
- `Sales.Customers`: Contains customer details.
- `Sales.CustomerTransactions`: Contains records of transactions made by customers.

## Columns
- `CustomerID`: Unique identifier for each customer from the `Sales.Customers` table.
- `CustomerName`: Name of the customer from the `Sales.Customers` table.
- `Sales.CustomerTransactions.CustomerID`: The identifier of customers who have made transactions, used in the subquery to exclude customers with any recorded transactions.

In [ ]:
USE WideWorldImporters

SELECT CustomerID, CustomerName
FROM Sales.Customers
WHERE CustomerID NOT IN (
    SELECT DISTINCT CustomerID
    FROM Sales.CustomerTransactions
);

___
___
___

# 9. `StockItems_With_Total_Qty_Ordered_Above_100.sql`

## Proposition
This query retrieves stock items that have been ordered in a total quantity greater than **100**, displaying their `StockItemID`, `StockItemName`, and the total quantity ordered.

## Tables
- `Warehouse.StockItems`: Contains details about stock items in the warehouse.
- `Sales.OrderLines`: Contains records of items included in customer orders.

## Columns
- `si.StockItemID`: Unique identifier for each stock item from the `Warehouse.StockItems` table.
- `si.StockItemName`: Name of the stock item from the `Warehouse.StockItems` table.
- `OrderTotals.StockItemID`: The identifier of stock items that have been ordered, used to join with the `Warehouse.StockItems` table and retrieved from a subquery that calculates the total quantity ordered for each stock item.
- `OrderTotals.TotalQuantityOrdered`: The total quantity of each stock item ordered, calculated by summing the `Quantity` from the `Sales.OrderLines` table.
- `Quantity`: The number of units ordered for each stock item, used in the subquery to calculate the total quantity ordered.


In [ ]:
USE WideWorldImporters

SELECT si.StockItemID, StockItemName, TotalQuantityOrdered
FROM Warehouse.StockItems si
JOIN (
    SELECT StockItemID, SUM(Quantity) AS TotalQuantityOrdered
    FROM Sales.OrderLines
    GROUP BY StockItemID
) AS OrderTotals ON si.StockItemID = OrderTotals.StockItemID
WHERE TotalQuantityOrdered > 100;

___
___
___

# 10. `Cust_Above_Avg_Transaction_Amounts.sql`

## Proposition
This query retrieves customers whose total transaction amounts exceed the average total transaction amounts across all customers, displaying their `CustomerID`, `CustomerName`, and `TotalPayments`.

## Tables
- `Sales.Customers`: Contains customer details.
- `Sales.CustomerTransactions`: Contains records of transactions made by customers.

## Columns
- `c.CustomerID`: Unique identifier for each customer from the `Sales.Customers` table.
- `c.CustomerName`: Name of the customer from the `Sales.Customers` table.
- `CustomerPaymentTotals.CustomerID`: The identifier of customers, retrieved from a subquery that sums the total transaction amounts for each customer.
- `CustomerPaymentTotals.TotalPayments`: The total amount of transactions for each customer, calculated by summing the `TransactionAmount` from the `Sales.CustomerTransactions` table.
- `TransactionAmount`: The amount associated with each customer transaction, used to calculate the total payments for each customer in the subquery.
- `PaymentAverages.TotalPayments`: The average of the total payments made by all customers, used in the outer query to filter customers whose total payments exceed the average.

In [ ]:
USE WideWorldImporters

SELECT c.CustomerID, CustomerName, TotalPayments
FROM Sales.Customers c
JOIN (
    SELECT CustomerID, SUM(TransactionAmount) AS TotalPayments
    FROM Sales.CustomerTransactions
    GROUP BY CustomerID
) AS CustomerPaymentTotals ON c.CustomerID = CustomerPaymentTotals.CustomerID
WHERE TotalPayments > (
    SELECT AVG(TotalPayments)
    FROM (
        SELECT SUM(TransactionAmount) AS TotalPayments
        FROM Sales.CustomerTransactions
        GROUP BY CustomerID
    ) AS PaymentAverages
);